# 고급 최적화 및 정규화 기법 - 실습 노트북 (고급 단계)

이 노트북은 고급 최적화 알고리즘과 정규화 기법을 구현하고 비교합니다.

## 목차
1. 고급 최적화 알고리즘 (Adam, RMSprop, AdaGrad)
2. 정규화 기법 (L1, L2, Dropout)
3. 배치 정규화 (Batch Normalization)
4. 종합 비교 및 분석

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

# 시드 설정
np.random.seed(42)

print("NumPy version:", np.__version__)

## 1. 고급 최적화 알고리즘

### 1.1 Adam Optimizer

**수식:**
$$m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t$$
$$v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2$$
$$\hat{m}_t = \frac{m_t}{1 - \beta_1^t}$$
$$\hat{v}_t = \frac{v_t}{1 - \beta_2^t}$$
$$\theta_t = \theta_{t-1} - \alpha \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}$$

**기호 설명:**
- $m_t$: 1차 모멘트 (평균)
- $v_t$: 2차 모멘트 (비중심 분산)
- $\beta_1, \beta_2$: 감쇠율 (일반적으로 0.9, 0.999)
- $\epsilon$: 수치 안정성 상수

In [ ]:
class AdamOptimizer:
    """Adam 최적화 알고리즘"""
    
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = {}  # 1차 모멘트
        self.v = {}  # 2차 모멘트
        self.t = 0   # 시간 단계
    
    def update(self, params, grads):
        """파라미터 업데이트"""
        self.t += 1
        
        for key in params.keys():
            # 초기화
            if key not in self.m:
                self.m[key] = np.zeros_like(params[key])
                self.v[key] = np.zeros_like(params[key])
            
            # 모멘트 업데이트
            self.m[key] = self.beta1 * self.m[key] + (1 - self.beta1) * grads[key]
            self.v[key] = self.beta2 * self.v[key] + (1 - self.beta2) * (grads[key] ** 2)
            
            # 편향 보정
            m_hat = self.m[key] / (1 - self.beta1 ** self.t)
            v_hat = self.v[key] / (1 - self.beta2 ** self.t)
            
            # 파라미터 업데이트
            params[key] -= self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
        
        return params

class RMSpropOptimizer:
    """RMSprop 최적화 알고리즘"""
    
    def __init__(self, learning_rate=0.001, beta=0.9, epsilon=1e-8):
        self.lr = learning_rate
        self.beta = beta
        self.epsilon = epsilon
        self.v = {}  # 2차 모멘트
    
    def update(self, params, grads):
        """파라미터 업데이트"""
        for key in params.keys():
            if key not in self.v:
                self.v[key] = np.zeros_like(params[key])
            
            self.v[key] = self.beta * self.v[key] + (1 - self.beta) * (grads[key] ** 2)
            params[key] -= self.lr * grads[key] / (np.sqrt(self.v[key]) + self.epsilon)
        
        return params

class SGDOptimizer:
    """기본 SGD (비교용)"""
    
    def __init__(self, learning_rate=0.01):
        self.lr = learning_rate
    
    def update(self, params, grads):
        """파라미터 업데이트"""
        for key in params.keys():
            params[key] -= self.lr * grads[key]
        return params

print("최적화 알고리즘 클래스 정의 완료")

### 1.2 최적화 알고리즘 비교

간단한 2D 함수로 각 최적화 알고리즘의 동작을 시각화합니다.

In [ ]:
# 최적화할 함수: Rosenbrock function (어려운 최적화 문제)
def rosenbrock(x, y, a=1, b=100):
    """Rosenbrock 함수"""
    return (a - x)**2 + b * (y - x**2)**2

def rosenbrock_grad(x, y, a=1, b=100):
    """Rosenbrock 함수의 그래디언트"""
    dx = -2 * (a - x) - 4 * b * x * (y - x**2)
    dy = 2 * b * (y - x**2)
    return np.array([dx, dy])

def optimize_function(optimizer, start_pos, num_iterations=100):
    """최적화 실행 및 경로 기록"""
    params = {'theta': np.array(start_pos, dtype=float)}
    path = [params['theta'].copy()]
    
    for _ in range(num_iterations):
        x, y = params['theta']
        grad = rosenbrock_grad(x, y)
        grads = {'theta': grad}
        
        params = optimizer.update(params, grads)
        path.append(params['theta'].copy())
    
    return np.array(path)

# 시작점
start = [-1.5, 2.5]

# 각 최적화 알고리즘 실행
path_sgd = optimize_function(SGDOptimizer(learning_rate=0.001), start)
path_rmsprop = optimize_function(RMSpropOptimizer(learning_rate=0.01), start)
path_adam = optimize_function(AdamOptimizer(learning_rate=0.01), start)

# 시각화
x = np.linspace(-2, 2, 100)
y = np.linspace(-1, 3, 100)
X, Y = np.meshgrid(x, y)
Z = rosenbrock(X, Y)

plt.figure(figsize=(14, 5))

# 등고선 플롯
plt.subplot(1, 2, 1)
plt.contour(X, Y, Z, levels=np.logspace(-1, 3, 20), cmap='viridis', alpha=0.6)
plt.plot(path_sgd[:, 0], path_sgd[:, 1], 'o-', label='SGD', markersize=3, linewidth=1.5)
plt.plot(path_rmsprop[:, 0], path_rmsprop[:, 1], 's-', label='RMSprop', markersize=3, linewidth=1.5)
plt.plot(path_adam[:, 0], path_adam[:, 1], '^-', label='Adam', markersize=3, linewidth=1.5)
plt.plot(1, 1, 'r*', markersize=15, label='Optimum')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Optimization Paths on Rosenbrock Function')
plt.legend()
plt.grid(True, alpha=0.3)

# 손실 수렴 비교
plt.subplot(1, 2, 2)
loss_sgd = [rosenbrock(p[0], p[1]) for p in path_sgd]
loss_rmsprop = [rosenbrock(p[0], p[1]) for p in path_rmsprop]
loss_adam = [rosenbrock(p[0], p[1]) for p in path_adam]

plt.semilogy(loss_sgd, label='SGD', linewidth=2)
plt.semilogy(loss_rmsprop, label='RMSprop', linewidth=2)
plt.semilogy(loss_adam, label='Adam', linewidth=2)
plt.xlabel('Iteration')
plt.ylabel('Loss (log scale)')
plt.title('Loss Convergence')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n최종 위치:")
print(f"  SGD:     {path_sgd[-1]}")
print(f"  RMSprop: {path_rmsprop[-1]}")
print(f"  Adam:    {path_adam[-1]}")
print(f"  최적값:  [1.0, 1.0]")

## 2. 정규화 기법

### 2.1 L2 정규화 (Weight Decay)

**수식:**
$$J = L + \frac{\lambda}{2} \sum_i w_i^2$$
$$w := (1 - \alpha\lambda)w - \alpha\frac{\partial L}{\partial w}$$

In [ ]:
# 기본 활성화 및 손실 함수
def sigmoid(z):
    return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return np.where(z > 0, 1, 0)

def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-10
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

def forward_pass(X, W1, b1, W2, b2, dropout_rate=0.0, training=True):
    """순방향 전파 (Dropout 포함)"""
    # 1층
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    
    # Dropout
    if training and dropout_rate > 0:
        D1 = (np.random.rand(*A1.shape) > dropout_rate).astype(float)
        A1 = A1 * D1 / (1 - dropout_rate)  # Inverted dropout
    else:
        D1 = np.ones_like(A1)
    
    # 2층
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    cache = {'X': X, 'Z1': Z1, 'A1': A1, 'D1': D1, 'Z2': Z2, 'A2': A2}
    return A2, cache

def backward_pass(Y, cache, W1, W2, l2_lambda=0.0, dropout_rate=0.0):
    """역전파 (L2 정규화 포함)"""
    m = Y.shape[1]
    
    # 출력층
    dZ2 = cache['A2'] - Y
    dW2 = (1/m) * np.dot(dZ2, cache['A1'].T) + (l2_lambda/m) * W2  # L2 정규화
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    
    # 은닉층
    dA1 = np.dot(W2.T, dZ2)
    if dropout_rate > 0:
        dA1 = dA1 * cache['D1'] / (1 - dropout_rate)  # Dropout 역전파
    dZ1 = dA1 * relu_derivative(cache['Z1'])
    dW1 = (1/m) * np.dot(dZ1, cache['X'].T) + (l2_lambda/m) * W1  # L2 정규화
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    
    return {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2}

print("정규화된 순방향/역방향 전파 함수 정의 완료")

### 2.2 정규화 효과 비교

과적합이 발생하기 쉬운 작은 데이터셋으로 정규화 효과를 확인합니다.

In [ ]:
# 작은 데이터셋 생성 (과적합 유도)
np.random.seed(42)
n_samples = 40  # 작은 데이터셋

X0 = np.random.randn(2, n_samples//2) * 0.5 - 1
Y0 = np.zeros((1, n_samples//2))

X1 = np.random.randn(2, n_samples//2) * 0.5 + 1
Y1 = np.ones((1, n_samples//2))

X_data = np.hstack([X0, X1])
Y_data = np.hstack([Y0, Y1])

# 데이터 섞기
shuffle_idx = np.random.permutation(X_data.shape[1])
X_data = X_data[:, shuffle_idx]
Y_data = Y_data[:, shuffle_idx]

print(f"데이터셋 크기: {X_data.shape}")

def train_with_regularization(X, Y, n_hidden=20, learning_rate=0.1, 
                             num_iterations=2000, l2_lambda=0.0, dropout_rate=0.0):
    """정규화를 포함한 학습"""
    np.random.seed(42)
    n_features = X.shape[0]
    
    # 파라미터 초기화
    W1 = np.random.randn(n_hidden, n_features) * 0.1
    b1 = np.zeros((n_hidden, 1))
    W2 = np.random.randn(1, n_hidden) * 0.1
    b2 = np.zeros((1, 1))
    
    losses = []
    
    for i in range(num_iterations):
        # 순방향 전파
        A2, cache = forward_pass(X, W1, b1, W2, b2, dropout_rate, training=True)
        
        # 손실 계산 (L2 정규화 포함)
        loss = binary_cross_entropy(Y, A2)
        if l2_lambda > 0:
            l2_reg = (l2_lambda / (2 * X.shape[1])) * (np.sum(W1**2) + np.sum(W2**2))
            loss += l2_reg
        losses.append(loss)
        
        # 역전파
        grads = backward_pass(Y, cache, W1, W2, l2_lambda, dropout_rate)
        
        # 파라미터 업데이트
        W1 -= learning_rate * grads['dW1']
        b1 -= learning_rate * grads['db1']
        W2 -= learning_rate * grads['dW2']
        b2 -= learning_rate * grads['db2']
    
    return {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}, losses

# 다양한 설정으로 학습
print("\n학습 중...")
params_none, losses_none = train_with_regularization(X_data, Y_data, l2_lambda=0.0)
params_l2, losses_l2 = train_with_regularization(X_data, Y_data, l2_lambda=0.1)
params_dropout, losses_dropout = train_with_regularization(X_data, Y_data, dropout_rate=0.3)
print("학습 완료!")

In [ ]:
# 학습 곡선 비교
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(losses_none, label='No Regularization', linewidth=2)
plt.plot(losses_l2, label='L2 Regularization (λ=0.1)', linewidth=2)
plt.plot(losses_dropout, label='Dropout (rate=0.3)', linewidth=2)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss with Different Regularizations')
plt.legend()
plt.grid(True, alpha=0.3)

# 가중치 크기 비교
plt.subplot(1, 2, 2)
w_norms = [
    np.linalg.norm(params_none['W1']),
    np.linalg.norm(params_l2['W1']),
    np.linalg.norm(params_dropout['W1'])
]
labels = ['No Reg', 'L2', 'Dropout']
plt.bar(labels, w_norms, color=['blue', 'green', 'orange'])
plt.ylabel('Weight Norm (L2)')
plt.title('Weight Magnitudes')
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\n최종 손실:")
print(f"  정규화 없음: {losses_none[-1]:.4f}")
print(f"  L2 정규화:   {losses_l2[-1]:.4f}")
print(f"  Dropout:     {losses_dropout[-1]:.4f}")
print(f"\n가중치 크기:")
print(f"  정규화 없음: {w_norms[0]:.4f}")
print(f"  L2 정규화:   {w_norms[1]:.4f}")
print(f"  Dropout:     {w_norms[2]:.4f}")

## 3. 배치 정규화 (Batch Normalization)

### 수식
$$\mu_B = \frac{1}{m} \sum_{i=1}^m x_i$$
$$\sigma_B^2 = \frac{1}{m} \sum_{i=1}^m (x_i - \mu_B)^2$$
$$\hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$$
$$y_i = \gamma \hat{x}_i + \beta$$

In [ ]:
def batch_normalization_forward(x, gamma, beta, epsilon=1e-5):
    """
    배치 정규화 순방향 전파
    
    Parameters:
    -----------
    x : (d, m) 입력
    gamma : (d, 1) 스케일 파라미터
    beta : (d, 1) 이동 파라미터
    
    Returns:
    --------
    out : 정규화된 출력
    cache : 역전파를 위한 캐시
    """
    # 배치 통계량
    mu = np.mean(x, axis=1, keepdims=True)
    var = np.var(x, axis=1, keepdims=True)
    
    # 정규화
    x_norm = (x - mu) / np.sqrt(var + epsilon)
    
    # 스케일 및 이동
    out = gamma * x_norm + beta
    
    cache = (x, x_norm, mu, var, gamma, beta, epsilon)
    return out, cache

def batch_normalization_backward(dout, cache):
    """
    배치 정규화 역전파
    """
    x, x_norm, mu, var, gamma, beta, epsilon = cache
    m = x.shape[1]
    
    # 그래디언트 계산
    dgamma = np.sum(dout * x_norm, axis=1, keepdims=True)
    dbeta = np.sum(dout, axis=1, keepdims=True)
    
    dx_norm = dout * gamma
    dvar = np.sum(dx_norm * (x - mu) * -0.5 * (var + epsilon)**(-1.5), axis=1, keepdims=True)
    dmu = np.sum(dx_norm * -1 / np.sqrt(var + epsilon), axis=1, keepdims=True) + \
          dvar * np.sum(-2 * (x - mu), axis=1, keepdims=True) / m
    
    dx = dx_norm / np.sqrt(var + epsilon) + dvar * 2 * (x - mu) / m + dmu / m
    
    return dx, dgamma, dbeta

# 배치 정규화 예제
print("=" * 70)
print("배치 정규화 수치 예제")
print("=" * 70)

# 입력 데이터
x = np.array([[2.0, 3.0, 4.0, 5.0],
              [1.0, 2.0, 3.0, 4.0]])
gamma = np.ones((2, 1))
beta = np.zeros((2, 1))

print(f"입력 x:\n{x}")
print(f"\n입력 통계:")
print(f"  평균: {np.mean(x, axis=1)}")
print(f"  표준편차: {np.std(x, axis=1)}")

# 배치 정규화 적용
out, cache = batch_normalization_forward(x, gamma, beta)

print(f"\n정규화된 출력:\n{out}")
print(f"\n출력 통계:")
print(f"  평균: {np.mean(out, axis=1)}")
print(f"  표준편차: {np.std(out, axis=1)}")
print("=" * 70)

## 4. 종합 비교 및 분석

모든 기법을 종합하여 비교합니다.

In [ ]:
# 결과 요약 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. 최적화 알고리즘 수렴 속도
axes[0, 0].semilogy(loss_sgd, label='SGD', linewidth=2)
axes[0, 0].semilogy(loss_rmsprop, label='RMSprop', linewidth=2)
axes[0, 0].semilogy(loss_adam, label='Adam', linewidth=2)
axes[0, 0].set_xlabel('Iteration')
axes[0, 0].set_ylabel('Loss (log scale)')
axes[0, 0].set_title('Optimizer Comparison')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. 정규화 효과
axes[0, 1].plot(losses_none, label='No Regularization', linewidth=2)
axes[0, 1].plot(losses_l2, label='L2', linewidth=2)
axes[0, 1].plot(losses_dropout, label='Dropout', linewidth=2)
axes[0, 1].set_xlabel('Iteration')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].set_title('Regularization Comparison')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. 가중치 분포 (정규화 전)
axes[1, 0].hist(params_none['W1'].flatten(), bins=30, alpha=0.7, label='No Reg')
axes[1, 0].hist(params_l2['W1'].flatten(), bins=30, alpha=0.7, label='L2')
axes[1, 0].set_xlabel('Weight Value')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Weight Distribution')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. 배치 정규화 효과
before_bn = x
after_bn = out
axes[1, 1].boxplot([before_bn[0], after_bn[0], before_bn[1], after_bn[1]], 
                    labels=['Before (dim 1)', 'After (dim 1)', 'Before (dim 2)', 'After (dim 2)'])
axes[1, 1].set_ylabel('Value')
axes[1, 1].set_title('Batch Normalization Effect')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("종합 분석 결과")
print("=" * 70)
print("\n1. 최적화 알고리즘:")
print("   - Adam이 가장 빠르게 수렴")
print("   - RMSprop도 SGD보다 훨씬 빠름")
print("   - SGD는 느리지만 때로는 더 나은 최종 성능")
print("\n2. 정규화:")
print("   - L2 정규화가 가중치 크기를 효과적으로 제한")
print("   - Dropout도 과적합 방지에 효과적")
print("   - 작은 데이터셋에서 특히 중요")
print("\n3. 배치 정규화:")
print("   - 입력을 정규화하여 학습 안정화")
print("   - 평균 0, 분산 1로 변환")
print("   - 더 높은 학습률 사용 가능")
print("=" * 70)

## 요약

이 노트북에서 다룬 내용:

1. **고급 최적화 알고리즘**
   - Adam: 모멘텀과 RMSprop의 결합
   - RMSprop: 적응적 학습률
   - 각 알고리즘의 수렴 특성 비교

2. **정규화 기법**
   - L2 정규화: 가중치 크기 제한
   - Dropout: 랜덤하게 뉴런 비활성화
   - 과적합 방지 효과 검증

3. **배치 정규화**
   - 미니배치 단위로 정규화
   - 학습 안정화 및 가속화
   - 순방향/역방향 전파 구현

### 실전 권장사항

- **최적화**: Adam을 기본으로 사용, 필요시 SGD로 fine-tuning
- **정규화**: L2와 Dropout을 함께 사용
- **배치 정규화**: 깊은 네트워크에서 필수적
- **학습률**: 배치 정규화 사용시 더 높은 학습률 가능

### 다음 단계

이제 실제 데이터셋(MNIST, CIFAR-10 등)에 이러한 기법들을 적용하여 실전 경험을 쌓으세요!